# Leveraging Sentiment Analysis for Enhanced Brand Management

## Problem Statement
---
In the rapidly evolving technology market, understanding customer sentiment towards products is crucial for companies like Google and Apple. These insights can guide product development, marketing strategies, customer service, and more. However, manually analysing customer sentiment is a time-consuming and labour-intensive process. Given the vast amount of customer feedback available on platforms like Twitter, it is virtually impossible for humans to process all the data in a timely manner.
Moreover, human analysis is subject to bias and inconsistency, and the quality of analysis can vary greatly depending on the individual’s understanding and interpretation. This makes it difficult to scale and standardize the sentiment analysis process.

Therefore, there is a need for an automated, efficient, and reliable solution to analyse customer sentiment towards Google and Apple products. Machine Learning, with its ability to learn patterns from large datasets and make predictions, offers a promising solution to this problem.
By applying Machine Learning techniques for sentiment analysis, we can process vast amounts of data in a fraction of the time it would take a human. This not only saves time and resources but also provides consistent and unbiased analysis. Furthermore, Machine Learning models can continuously learn and improve over time, adapting to new trends and nuances in customer sentiment.

## Business Understanding
---
Twitter is a platform where users often share their experiences and opinions about products. Analysing these sentiments can provide valuable feedback on what users like or dislike about a product, which can guide improvements and new features. Sentiment analysis can help understand how the brand is perceived in the market. Positive sentiment is usually associated with a strong brand image, while negative sentiment can indicate potential issues that need to be addressed.  By analysing sentiment, Apple and Google can identify trends in consumer behaviour and preferences. This can inform strategic decisions, such as the timing of product releases or marketing campaigns.

Comparing sentiment towards different products can provide insights into competitive positioning. For example, if sentiment towards an Apple product is more positive than a similar Google product, it might indicate a competitive advantage for Apple. Negative tweets can be a signal of customer service issues that need to be addressed. Apple and Google can use sentiment analysis to proactively identify and resolve these issues. This data-driven approach fosters brand loyalty, increases customer satisfaction, and ultimately drives sales growth.

### Key Objectives
$i.$ Utilize Natural Language Processing techniques to construct a machine learning model for automated sentiment analysis of tweets related to Google and Apple products.<br>
$ii.$ Evaluate and select the most suitable machine learning model for sentiment analysis based on its performance metrics.<br>
$iii.$ Analyse frequency of the sentiments expressed in tweets about Google and Apple products.

## Data Understanding
---
The dataset, sourced from CrowdFlower via data.world, comprises over 9,000 tweets with sentiment ratings labeled as positive, negative, or neutral by human raters. The dataset contains three columns:

$i.$ `tweet_text`: This column contains the text of the tweet, facilitating sentiment analysis based on the content itself.

$ii.$ `emotion_in_tweet_is_directed_at`: This column indicates whether the expressed emotion pertains to a specific brand or product. It enables targeted sentiment analysis tailored to the brand's performance.

$iii.$ `is_there_an_emotion_directed_at_a_brand_or_product (target variable)`: This column serves as a quick indicator of brand-related sentiment, allowing for efficient initial filtering of relevant data.

## Exploratory Data Analysis
---

## Data Preprocessing And Modeling